In [3]:
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v2
from datagenerator import ImageDataGenerator
from tensorflow.contrib.data import Iterator
import numpy as np
from datetime import datetime
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Path to the textfiles for the trainings and validation set
num = 0
train_file = '/HS_code/0_Code/train%d_HR_ICROS.txt'%num
val_file1 = '/HS_code/0_Code/valid%d_HR_ICROS.txt'%num
batch_size = 64
num_classes = 2
learning_rate = 0.001
training_epochs = 50
display_step = 20
layer_num = 101

# Path for tf.summary.FileWriter and to store model checkpoints
filewriter_path = "/HS_code/2_Result_TB/tensorboard_HS_%s_ResNet_%d_hr%d_valid%d_%depoch_0_001"%(date,layer_num,num,num,num_epochs)
checkpoint_path = "/HS_code/1_Model_CP/checkpoints_HS_%s_ResNet_%d_hr%d_valid%d_%depoch_0_001"%(date,layer_num,num,num,num_epochs)

NameError: name 'date' is not defined

In [ ]:
with tf.device('/gpu:0'):
    # data load
    tr_data = ImageDataGenerator(train_file,
                                 mode='training',
                                 batch_size=batch_size,
                                 num_classes=num_classes,
                                 shuffle=True)
    val_data1 = ImageDataGenerator(val_file1,
                                  mode='inference',
                                  batch_size=batch_size,
                                  num_classes=num_classes,
                                  shuffle=False)
    
     # create an reinitializable iterator given the dataset structure
    iterator = Iterator.from_structure(tr_data.data.output_types,
                                           tr_data.data.output_shapes)

    next_batch = iterator.get_next()

    
    # Ops for initializing the two different iterators
    training_init_op = iterator.make_initializer(tr_data.data)
    validation_init_op1 = iterator.make_initializer(val_data1.data)
    
    # TF placeholder for graph input and output
    x = tf.placeholder(tf.float32, [batch_size, 227, 227, 3])
    y = tf.placeholder(tf.float32, [batch_size, num_classes])

    
    net, net_points = resnet_v2.resnet_v2_101(x, 
                                              num_classes=2,
                                              is_training=True)
    
    # Op for calculating the loss
    with tf.name_scope("cross_ent"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=y))
        cost = tf.reduce_mean(loss)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    # Add the loss to summary
    tf.summary.scalar('cross_entropy', loss)
    
    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(net[:,0,0], 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # Add the accuracy to the summary
    tf.summary.scalar('accuracy', accuracy)
    
    # Initialize the FileWriter
    writer = tf.summary.FileWriter(filewriter_path)

    # Merge all summaries together
    merged_summary = tf.summary.merge_all()
    
    train_batches_per_epoch = int(np.floor(tr_data.data_size / batch_size))
    val_batches_per_epoch1 = int(np.floor(val_data1.data_size / batch_size)) 
    print(train_batches_per_epoch)

In [4]:
# Training
config=tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer())
    print("{} Start training...".format(datetime.now()))
    img_batch = np.zeros((batch_size,227,227,3), dtype ='uint8')
    for epoch in range(training_epochs):
        print("{} Epoch number: {}".format(datetime.now(), epoch+1))
        # Initialize iterator with the training dataset
        sess.run(training_init_op)

        for step in range(train_batches_per_epoch):

            # get next batch of data
            img_batch, label_batch = sess.run(next_batch)       

            # And run the training op
            sess.run(optimizer, feed_dict={x: img_batch,
                                          y: label_batch})

            # Generate summary with the current batch of data and write to file
            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={x: img_batch,
                                                        y: label_batch})
                print("{} {} step".format(datetime.now(), step))

        # Validate the model on the entire validation set
        print("{} Start validation".format(datetime.now()))
        sess.run(validation_init_op1)
        test_acc = 0.
        test_count = 0
        
        for a in range(val_batches_per_epoch1):

            img_batch, label_batch = sess.run(next_batch)
            acc = sess.run(accuracy, feed_dict={x: img_batch,
                                                y: label_batch})
            test_acc += acc
            test_count += 1

        test_acc /= test_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
            

2018-08-08 10:38:44.074517 Start training...
2018-08-08 10:38:44.075823 Epoch number: 1
2018-08-08 10:38:49.030325 0 step
2018-08-08 10:38:59.638311 20 step
2018-08-08 10:39:10.291200 40 step
2018-08-08 10:39:18.645710 Start validation
2018-08-08 10:39:21.901815 Validation Accuracy = 0.7846
2018-08-08 10:39:21.901985 Epoch number: 2
2018-08-08 10:39:24.202608 0 step
2018-08-08 10:39:35.056884 20 step
2018-08-08 10:39:46.099766 40 step
2018-08-08 10:39:54.660486 Start validation
2018-08-08 10:39:57.709423 Validation Accuracy = 0.7701
2018-08-08 10:39:57.709858 Epoch number: 3
2018-08-08 10:40:00.079238 0 step
2018-08-08 10:40:10.821630 20 step
2018-08-08 10:40:21.765315 40 step
2018-08-08 10:40:30.290548 Start validation
2018-08-08 10:40:33.331883 Validation Accuracy = 0.7946
2018-08-08 10:40:33.332067 Epoch number: 4
2018-08-08 10:40:35.698042 0 step
2018-08-08 10:40:46.738278 20 step
2018-08-08 10:40:57.730187 40 step
2018-08-08 10:41:06.222099 Start validation
2018-08-08 10:41:09.272

2018-08-08 10:58:30.077852 0 step
2018-08-08 10:58:40.879837 20 step
2018-08-08 10:58:51.823136 40 step
2018-08-08 10:59:00.266735 Start validation
2018-08-08 10:59:03.330930 Validation Accuracy = 0.9598
2018-08-08 10:59:03.331131 Epoch number: 35
2018-08-08 10:59:05.751020 0 step
2018-08-08 10:59:16.594464 20 step
2018-08-08 10:59:27.653714 40 step
2018-08-08 10:59:36.109201 Start validation
2018-08-08 10:59:39.119597 Validation Accuracy = 0.9788
2018-08-08 10:59:39.119728 Epoch number: 36
2018-08-08 10:59:41.494291 0 step
2018-08-08 10:59:52.511691 20 step
2018-08-08 11:00:03.510754 40 step
2018-08-08 11:00:11.956188 Start validation
2018-08-08 11:00:14.962992 Validation Accuracy = 0.9699
2018-08-08 11:00:14.963188 Epoch number: 37
2018-08-08 11:00:17.347349 0 step
2018-08-08 11:00:28.297686 20 step
2018-08-08 11:00:39.170882 40 step
2018-08-08 11:00:47.918351 Start validation
2018-08-08 11:00:51.007764 Validation Accuracy = 0.9844
2018-08-08 11:00:51.007868 Epoch number: 38
2018-08-